<a href="https://colab.research.google.com/github/efekaanefe/Rapper-AI/blob/main/rapper_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torch.autograd import Variable
import torch.nn.functional as F


In [2]:
with open('eminem-lyrics.txt', 'r') as f:
    text = f.read()

words = text.split()

len(words)

102390

## Word level inputs and targets

In [47]:
words = text.split()
unique_words = sorted(list(set(words)))
vocab_size = len(unique_words)
print(f'Unique words/vocabulary size: {vocab_size}')

# Create mappings from words to indices and vice versa
word_to_idx = {word: idx for idx, word in enumerate(unique_words)}
idx_to_word = {idx: word for idx, word in enumerate(unique_words)}

# Encode the text using word-level vocabulary
encoded_text = np.array([word_to_idx[word] for word in words])

seq_length = 10  # Adjust sequence length as needed for words
num_samples = len(encoded_text) // seq_length

input_sequences = []
target_sequences = []

# The fix: Ensure all sequences are of the same length by padding shorter ones
for i in range(num_samples):
    start_idx = i * seq_length
    end_idx = start_idx + seq_length

    input_seq = encoded_text[start_idx:end_idx]
    target_seq = encoded_text[start_idx + 1:end_idx + 1]

    # Pad if the sequence is shorter than seq_length
    if len(input_seq) < seq_length:
        input_seq = np.pad(input_seq, (0, seq_length - len(input_seq)), 'constant', constant_values=0)  # Pad with zeros
    if len(target_seq) < seq_length:
        target_seq = np.pad(target_seq, (0, seq_length - len(target_seq)), 'constant', constant_values=0)  # Pad with zeros

    input_sequences.append(input_seq)
    target_sequences.append(target_seq)

input_sequences = torch.tensor(np.array(input_sequences), dtype=torch.long)
target_sequences = torch.tensor(np.array(target_sequences), dtype=torch.long)

print(input_sequences.shape, target_sequences.shape, encoded_text.shape)

print(f"1st input sequence: \n{input_sequences[0]}", end="\n\n")
print("Input sequence as words:")
print(' '.join([idx_to_word[int(i)] for i in input_sequences[0]]), end="\n\n")
print("Target sequence as words:")
print(' '.join([idx_to_word[int(i)] for i in target_sequences[0]]))


Unique words/vocabulary size: 14734
torch.Size([10239, 10]) torch.Size([10239, 10]) (102390,)
1st input sequence: 
tensor([ 2823, 14612, 13434,  8898,  1632,  4696,  4703, 13907,  8759, 13351])

Input sequence as words:
Oh yeah, this is Eminem baby, back up in that

Target sequence as words:
yeah, this is Eminem baby, back up in that motherfucking


## Character level inputs and targets

In [42]:
# Create a character-level vocabulary
chars = sorted(list(set(text)))
char_size = len(chars)
print(f'Unique characters size: {char_size}')

# Create mappings from characters to indices and vice versa
char_to_idx = {char: idx for idx, char in enumerate(chars)}
idx_to_char = {idx: char for idx, char in enumerate(chars)}

encoded_text = np.array([char_to_idx[char] for char in text])

seq_length = 100 # Sequence length
num_samples = len(encoded_text) // seq_length

input_sequences = []
target_sequences = []

for i in range(num_samples):
    start_idx = i * seq_length
    end_idx = start_idx + seq_length
    input_sequences.append(encoded_text[start_idx:end_idx])
    target_sequences.append(encoded_text[start_idx + 1:end_idx + 1])

input_sequences = torch.tensor(np.array(input_sequences), dtype=torch.long)
target_sequences = torch.tensor(np.array(target_sequences), dtype=torch.long)

print(input_sequences.shape, target_sequences.shape, encoded_text.shape)

print(f"1st input sequence: \n{input_sequences[0]}", end="\n\n")
print("Input sequence as words:")
print(''.join([idx_to_char[int(i)] for i in input_sequences[0]]), end="\n\n")
print("Target sequence as words:")
print(''.join([idx_to_char[int(i)] for i in target_sequences[0]]))


Unique characters size: 95
torch.Size([5225, 100]) torch.Size([5225, 100]) (522527,)
1st input sequence: 
tensor([44, 66,  1, 83, 63, 59, 66, 13,  1, 78, 66, 67, 77,  1, 67, 77,  1, 34,
        71, 67, 72, 63, 71,  1, 60, 59, 60, 83, 13,  1, 60, 59, 61, 69,  1, 79,
        74,  1, 67, 72,  1, 78, 66, 59, 78,  1, 71, 73, 78, 66, 63, 76, 64, 79,
        61, 69, 67, 72, 65,  1, 59, 77, 77,  0, 44, 72, 63,  1, 78, 67, 71, 63,
         1, 64, 73, 76,  1, 83, 73, 79, 76,  1, 71, 73, 78, 66, 63, 76,  1, 64,
        79, 61, 69, 67, 72, 65,  1, 71, 67, 72])

Input sequence as words:
Oh yeah, this is Eminem baby, back up in that motherfucking ass
One time for your mother fucking min

Target sequence as words:
h yeah, this is Eminem baby, back up in that motherfucking ass
One time for your mother fucking mind


## Model

In [33]:
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        x = self.embedding(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out.reshape(out.size(0) * out.size(1), out.size(2)))
        return out, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        return (weight.new_zeros(num_layers, batch_size, hidden_size),
                weight.new_zeros(num_layers, batch_size, hidden_size))


## Training

In [49]:
# Hyperparameters
embed_size = 512
hidden_size = 256
num_layers = 2
num_epochs = 1
learning_rate = 0.001
batch_size = 64

model = LSTMModel(vocab_size, embed_size, hidden_size, num_layers)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [50]:
# Training loop
for epoch in range(num_epochs):
    hidden = model.init_hidden(batch_size)
    total_loss = 0

    for i in range(0, input_sequences.size(0) - batch_size, batch_size):
        inputs = input_sequences[i:i+batch_size]
        targets = target_sequences[i:i+batch_size]

        hidden = tuple([h.detach() for h in hidden])

        # Forward
        outputs, hidden = model(inputs, hidden)
        loss = criterion(outputs, targets.view(-1))

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / (input_sequences.size(0) // batch_size)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}')


Epoch [1/1], Loss: 7.6352
